# Process and barcode sequencing reads
This script takes paired read (read 1: barcode + UMI (27bp), read 2: staggers + UMI + partial barcode (49bp)) fastq files and does the following:
1. trims read 2 adapter sequences to recover barcode+UMI sequence
2. merges read 1 and 2 with FLASH
3. identifies and counts barcodes

In [1]:
import glob
import os
import subprocess
import regex
import gzip
from Bio import SeqIO
import pandas as pd
import multiprocessing as mp
import numpy as np

# check number of available cores
len(os.sched_getaffinity(0))

20

## Inspect fastq files for quality control using FastQC

In [2]:
# !mkdir ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/fastqc/
# !fastqc -q -o ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/fastqc/ ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/18146FL-40*fastq.gz
# !mkdir ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run2/fastqc/
# !fastqc -q -o ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run2/fastqc/ ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run2/18146FL-40*fastq.gz
# !mkdir ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run3/fastqc/
# !fastqc -q -o ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run3/fastqc/ ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run3/18146FL-40*fastq.gz
# !mkdir ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run4/fastqc/
# !fastqc -q -o ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run4/fastqc/ ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run4/18146FL-40*fastq.gz
# !mkdir ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run5/fastqc/
# !fastqc -q -o ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run5/fastqc/ ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run5/18146FL-40*fastq.gz

## Summarize FastQC output with MultiQC

In [3]:
# !multiqc -o ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/fastqc/ ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/fastqc/
# !multiqc -o ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run2/fastqc/ ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run2/fastqc/
# !multiqc -o ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run3/fastqc/ ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run3/fastqc/
# !multiqc -o ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run4/fastqc/ ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run4/fastqc/
# !multiqc -o ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run5/fastqc/ ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run5/fastqc/

## Map fastq file names to sample names

In [4]:
# key to map fastq names to output names
seqID_to_sampleName = {}
sample_key_file = "/home/users/rang/scratch/yeast/crispey3/gxg_5comp_aug2021/SampleKey-18146-40.txt"
with open(sample_key_file, 'r') as sample_key:
    sample_key.readline() # skip header
    for line in sample_key:
        seqID, sampleName = line.rstrip().split("\t")
        sampleName = sampleName.replace("_","-")
        seqID_to_sampleName[seqID] = sampleName


## Trim read 2 adapters with cutadapt
Remove staggers, leaving the UMI+partial barcode sequence (19-26bp, may be shorter depending on quality trimming)

In [5]:
# working directory with fastq files
working_dir="/home/users/rang/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/"
os.chdir(working_dir)

# get read 2 files for trimming
fastq_list = sorted([os.path.abspath(x) for x in glob.glob("./run*/18146FL-40*R2_001.fastq.gz")])

In [6]:
# cutadapt parameters to trim read 2 to get barcode+UMI (27bp)
adapter_5prime = 'GGCCAGTTTAAACTT'
adapter_3prime = 'GCATGGC'

num_of_cores = len(os.sched_getaffinity(0))
err = 0.2 # fraction tolerated for adapter matching
min_r2_length = 12 # R2 must contain at least UMI sequence and some of SphI linker
output_dir_name = 'trimmed'

In [7]:
# store sample key in regex pattern
pattern = regex.compile('|'.join(seqID_to_sampleName.keys()))

# trim read 2, filter untrimmed read pairs
for fastq_path in fastq_list:
    fastq_dir = os.path.dirname(fastq_path)
    output_dir = fastq_dir + "/"+output_dir_name+"/"
    os.makedirs(output_dir, exist_ok=True)
    
    # rename output files by sample key stored in seqID_to_sampleName 
    fastq_file = os.path.basename(fastq_path)
    output_file_r2 = pattern.sub(lambda x: seqID_to_sampleName[x.group()], fastq_file).replace("_001.fastq.gz", "_001_trimmed.fastq.gz")
    output_file_r1 = output_file_r2.replace("_R2_", "_R1_")
    
    cutadapt_cmd = ["cutadapt", "-g", adapter_5prime+"..."+adapter_3prime, #adapter_5prime, 
                    "-j", str(num_of_cores), 
                    "-e", str(err), 
                    #"-q", "20", # use -q for miseq/hiseq quality trimming
                    "--nextseq-trim", "20", # use this option for nextseq quality trimming
                    "--discard-untrimmed", "-m", str(min_r2_length), 
                    "--pair-filter=first", 
                    "-o", output_dir+output_file_r2, "-p", output_dir+output_file_r1,
                    fastq_path, fastq_path.replace("L001_R2_", "L001_R1_")]
    
    subprocess.run(cutadapt_cmd)
  
print('Done trimming!')

Done trimming!


## Merge read 1 and read 2 with FLASH to produce final barcode+UMI sequence

In [8]:
working_dir="/home/users/rang/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/"
os.chdir(working_dir)

# get files for merging
fastq_list = sorted([os.path.abspath(x) for x in glob.glob("./run*/trimmed/*R1_001_trimmed.fastq.gz")])

In [9]:
# FLASH parameters
min_overlap = 8 # cannot be longer than the shorter read.
max_overlap = 12 # cannot be longer than the shorter read.
max_mismatch = 0.25 #default is 0.25, can set 0.4 to tolerate lower seq quality
num_of_cores = len(os.sched_getaffinity(0))
output_dir_name = 'merged'

In [10]:
# use FLASH to merge trimmed-filtered read 2 and read 1 data to produce final 27bp sequence containing barcode and UMI data
for fastq_path in fastq_list:
    fastq_dir = os.path.dirname(fastq_path)
    output_dir = fastq_dir + "/"+output_dir_name+"/"
    os.makedirs(output_dir, exist_ok=True)
    
    # check output file naming 
    output_prefix = os.path.basename(fastq_path).split("_")[0]+"_barcode"
    
    flash_cmd = ["flash", "-m", str(min_overlap), "-M", str(max_overlap),
                 "-x", str(max_mismatch), #"-O", # use -O if innie-only merging does not work
                 "-t", str(num_of_cores),
                 "-o", output_prefix, "-d", output_dir, 
                 "--compress", 
                 fastq_path, fastq_path.replace("_R1_", "_R2_")]
    subprocess.run(flash_cmd)

print('Done merging!')

Done merging!


## (optional) Join processed fastq files across multiple sequencing runs for each sample

In [11]:
# !mkdir ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/final/
# !for file in ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/merged/*.extendedFrags.fastq.gz; do
# !cat ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run*/trimmed/merged/$(basename "$file") > ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/final/$(basename "$file")
# !done

## Count barcodes
Counting barcodes consists of several steps. First, parse each fastq file and count all sequences. After assembling into an initial sequences counts matrix, extract the barcode and UMI sequences and map them to a reference table of barcodes and UMIs. Counts for ID-able sequences are onsolidated into a final counts matrix for input to DESeq2.

It is possible to parallelize this process if handling multiple samples across different conditions. However, it is more efficient to put all counts across all conditions into a single matrix and run the barcode mapping a single time, since the barcodes are the same across all conditions.

In [12]:
def count_seqs(fastq_file, min_seq_length, max_seq_length):
    '''
    Parses a fastq file and counts sequences. Returns dict of counts
    '''
    seq_counts_dict = {}
    # parse fastq
    with gzip.open(fastq_file, 'rt') as fastq:
        for read in SeqIO.parse(fastq, "fastq"):
            # filter for sequences within min/max length and contains no N's
            if min_seq_length <= len(read.seq) <= max_seq_length and read.seq.count("N")==0:
                sequence = str(read.seq)
                # count sequence
                try:
                    seq_counts_dict[sequence] += 1
                except KeyError:
                    seq_counts_dict[sequence] = 1
    
    return seq_counts_dict

    
def map_seq_to_barcode_umi(seq, barcode_table, umi_list, barcode_length, umi_length, linker_seq):
    '''
    splits a sequence into barcode and UMI, maps to barcode table and UMI list to assign ID
    does NOT do UMI mapping if umi_list is empty.
    '''
    final_id = 'UNKNOWN'
    
    barcode, umi = split_barcode_umi_from_seq(seq, barcode_length, umi_length, linker_seq)
    
    if len(barcode)<barcode_length/2 or len(umi)<umi_length/2:
        # Barcode/UMI too short
        return final_id

    # assign barcode ID
    barcode_id = assign_barcode(barcode=barcode, barcode_table=barcode_table, error=1)
    if barcode_id == 'UNKNOWN':
        # Barcode cannot be identified
        return barcode_id
    else:
        # add barcode ID to final ID
        final_id = barcode_id

    # assign UMI ID (if applicable)
    if len(umi_list)>0:
        umi_id = assign_umi(umi=umi, umi_list=umi_list, error=1)
        if umi_id == 'UNKNOWN':
            # UMI cannot be identified
            return umi_id
        else:
            # add UMI ID to final ID
            final_id = '-'.join([final_id, str(umi_id)])
    
    return final_id
    

def split_barcode_umi_from_seq(seq, barcode_length, umi_length, linker_seq):
    '''
    Splits seq by linker_seq and returns barcode and UMI sequence
    If linker seq cannot be found (e.g. sequencing error) or yields multiple splits,
    fall back to splitting by base position.
    '''
    barcode = ''
    umi = ''
    
    try:
        # split by linker
        barcode, umi = seq.split(linker_seq) # can try error tolerant regex?
    except ValueError:
        # split by base position
        if barcode_length>0 and umi_length==0:
            barcode = seq[:barcode_length]
        else:
            umi = seq[-umi_length:]
            barcode = seq[:-(umi_length+len(linker_seq))] # may return partial barcodes for short sequences

    return (barcode, umi)


def assign_barcode(barcode, barcode_table, error):
    '''
    Searches barcode table for barcode sequence and returns unique barcode ID
    Tries perfect match first, then error-tolerant regex
    '''
    if barcode == '':
        barcode_id = 'UNKNOWN'
    else:
        try:
            # search for perfect match
            barcode_id = barcode_table.loc[barcode, 'Unique_ID']
        except KeyError:
            # search by error-tolerant regex
            pattern = "(?:"+barcode+"){s<="+str(error)+"}"
            search = [bool(regex.search(pattern, x)) for x in barcode_table.index]
            if sum(search)==1:
                barcode_id = barcode_table.loc[search, 'Unique_ID'][0]
            else:
                # barcode cannot be identified
                barcode_id = 'UNKNOWN'
    
    return barcode_id


def assign_umi(umi, umi_list, error):
    '''
    Searches umi list for umi sequence and returns 1-index position as umi ID
    Tries perfect match first, then error-tolerant regex
    '''
    if umi == '':
        umi_id = 'UNKNOWN'
    else:
        try:
            # search for perfect match
            umi_id = umi_list.index(umi)+1
        except ValueError:
            # search by error-tolerant regex
            pattern = "(?:"+umi+"){s<="+str(error)+"}"
            search = [bool(regex.search(pattern, x)) for x in umi_list]
            if sum(search) == 1:
                umi_id = search.index(True)+1
            else:
                # UMI cannot be identified
                umi_id = 'UNKNOWN'
    
    return umi_id


In [13]:
working_dir="/home/users/rang/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/final/"
os.chdir(working_dir)

#output directory
output_dir = "/home/users/rang/scratch/yeast/crispey3/gxg_5comp_aug2021/counts/"

# merged reads to count barcodes from
fastq_list = sorted(glob.glob("*extendedFrags.fastq.gz")) # check for file name
# sample names for each fastq
sample_name_list = [fastq_file.split("_")[0] for fastq_file in fastq_list] # adjust accordingly to generate sample name for counts matrix

# sequence counts file (before combining)
seq_counts_filename = "seq_counts.txt"

# mapped barcode-UMI counts file
barcode_counts_filename = "barcode_counts.txt"


# open barcode reference file
barcode_reference_file = '/home/users/rang/crispey3/library_design/Input/12BP_PBCs_well_grouped.csv'
barcode_table = pd.read_csv(barcode_reference_file, index_col=1)

# approved list of UMIs used in cloning CRISPEY3 plasmid
umi_list = ['ACGCGTGAA',
            'ATGTGGCTC',
            'CAGAGGATC',
            'CTGTGGCAA',
            'GTGTGATTC',
            'TAGAGGACT',] # only first 6 UMIs were included in cloning CRISPEY3 libaries
#             'AAGAGCCTC',
#             'AAGAGGAGG',
#             'ATGTGCGAA',
#             'ATGTGTAGG',
#             'CAGAGCCAA',
#             'CTGTGATGG',
#             'CTGTGTATC',
#             'GAGAGGAAA',
#             'TCGCGGTAA',
#             'TTGTGCGTC']
umi_list = sorted(umi_list)


In [14]:
# count sequences in each fastq file
fastq_dict = dict(zip(sample_name_list, fastq_list))
with mp.Pool(min(len(os.sched_getaffinity(0)), len(fastq_list))) as pool:
    seq_counts_df = {sample_name : pool.apply_async(count_seqs, (fastq_file, 20, 27)) for sample_name, fastq_file in fastq_dict.items()}
    seq_counts_df = {sample_name : res.get() for sample_name, res in seq_counts_df.items()}
    
# merge to dataframe
seq_counts_df = pd.DataFrame.from_dict(seq_counts_df, orient="columns")
seq_counts_df.index.name = 'sequence'

# write to file to inspect
os.makedirs(output_dir, exist_ok=True)
seq_counts_df.to_csv(output_dir+seq_counts_filename, sep="\t")


In [15]:
# seq_counts_df = pd.read_csv(output_dir+seq_counts_filename, sep="\t", index_col='sequence')

# map each sequence in seq_counts_df to barcode-UMI ID
mapped_counts_df = seq_counts_df.copy()
mapped_counts_df = mapped_counts_df.reset_index()

# filter out low count barcodes (computationally expensive to map these rare barcodes, minimal impact to total count (~2%))
cutoff = 15
mapped_counts_df = mapped_counts_df.loc[mapped_counts_df.sum(axis=1)>=cutoff]

# do multiprocessing for sequence mapping
num_of_cores = len(os.sched_getaffinity(0))
with mp.Pool(num_of_cores) as pool:
    barcode_umi_id_list = [pool.apply_async(map_seq_to_barcode_umi, (seq, barcode_table, umi_list, 12, 9, 'GCATGC')) for seq in mapped_counts_df['sequence']]
    barcode_umi_id_list = [res.get() for res in barcode_umi_id_list]
    
# mapped_counts_df['barcode_umi_id'] = [x for res in barcode_umi_id_lists for x in res] #pd.concat(barcode_umi_id_lists)
mapped_counts_df['barcode_umi_id'] = barcode_umi_id_list
display(mapped_counts_df)

# remove unknowns
mapped_counts_df = mapped_counts_df.query('barcode_umi_id!="UNKNOWN"')

# consolidate counts
mapped_counts_df = mapped_counts_df.groupby('barcode_umi_id').sum().fillna(0).astype(int)
mapped_counts_df.index.name = 'barcode'
display(mapped_counts_df)

# write all counts to output file
os.makedirs(output_dir, exist_ok=True)
mapped_counts_df.to_csv(output_dir+barcode_counts_filename, sep="\t")


# # one-liner to map sequences to barcode-umi IDs
# # warning: single-threaded, expected to be slow
# mapped_counts_df['barcode_umi_id'] = mapped_counts_df['sequence'].apply(map_seq_to_barcode_umi, args=(barcode_table, umi_list, 12, 9, 'GCATGC'))


,sequence,BYA-t1-1,BYA-t1-2,BYA-t1-3,BYA-t10-1,BYA-t10-2,BYA-t10-3,BYA-t13-1,BYA-t13-2,BYA-t13-3,...,YPS-t13-1,YPS-t13-2,YPS-t13-3,YPS-t4-1,YPS-t4-2,YPS-t4-3,YPS-t7-1,YPS-t7-2,YPS-t7-3,barcode_umi_id
0,ATTCACTTAAGCGCATGCCAGAGGATC,1254.0,1193.0,1253.0,945.0,993.0,931.0,687.0,808.0,703.0,...,483.0,371.0,429.0,309.0,298.0,394.0,398.0,416.0,205.0,053_046-3
1,CACCGTCTCCGTGCATGCATGTGGCTC,2093.0,2090.0,2194.0,2706.0,2405.0,2109.0,2736.0,2279.0,1874.0,...,1907.0,2385.0,2457.0,2106.0,2056.0,1979.0,2130.0,2198.0,1309.0,077_044-2
2,ATTCGAGTGCTCGCATGCATGTGGCTC,7210.0,7233.0,7354.0,7175.0,7027.0,6595.0,6471.0,6182.0,5985.0,...,3628.0,2925.0,2947.0,3672.0,3430.0,3318.0,3577.0,3304.0,1878.0,037_113-2
3,CAAAGCGCTCACGCATGCATGTGGCTC,17745.0,17520.0,17476.0,16406.0,18203.0,15915.0,14200.0,16655.0,14952.0,...,14951.0,14606.0,13904.0,15027.0,14376.0,14348.0,14687.0,15201.0,8500.0,053_101-2
4,ATACTCAACAACGCATGCCTGTGGCAA,1558.0,1597.0,1604.0,1552.0,1362.0,1734.0,1397.0,1119.0,1684.0,...,1556.0,1803.0,1665.0,1505.0,1616.0,1714.0,1434.0,1715.0,931.0,057_118-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6241535,CTACGGCGGAGAGCATGCACGCGTGAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.0,NaN,NaN,1.0,NaN,NaN,12.0,NaN,191_082-1
6243849,ACAGCGGCATGCCTGTGGCAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,70.0,NaN,NaN,5.0,NaN,NaN,6.0,NaN,UNKNOWN
6244852,TGACACCGCGGTGCATGCATGTGGCTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,21.0,NaN,NaN,6.0,NaN,NaN,10.0,NaN,045_083-2
6287759,ACTGAAGGATTGGCATGCATGTGGTTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,11.0,NaN,NaN,NaN,041_004-2


,BYA-t1-1,BYA-t1-2,BYA-t1-3,BYA-t10-1,BYA-t10-2,BYA-t10-3,BYA-t13-1,BYA-t13-2,BYA-t13-3,BYA-t4-1,...,YPS-t10-3,YPS-t13-1,YPS-t13-2,YPS-t13-3,YPS-t4-1,YPS-t4-2,YPS-t4-3,YPS-t7-1,YPS-t7-2,YPS-t7-3
barcode,,,,,,,,,,,,,,,,,,,,,
001_003-3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001_004-3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001_022-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001_022-3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001_022-4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ladder_043-2,1,10,17,1,2,2,2,2,0,8,...,0,0,0,0,0,0,0,0,0,0
Ladder_043-3,8,16,11,2,8,6,3,3,5,3,...,0,0,0,0,0,0,2,0,0,0
Ladder_043-4,3,4,2,0,0,3,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0


## (optional) Combine counts across UMIs per barcode
The counts of different UMIs of the same barcode can be added together to produce a stacked counts matrix

In [16]:
# combine counts from different UMIs of the same barcode
stacked_counts_filename = "stacked_barcode_counts.txt"

stacked_counts_df = mapped_counts_df.groupby(by=lambda x: x.split('-')[0]).sum()
stacked_counts_df.index.name = 'barcode'
stacked_counts_df.to_csv(output_dir+stacked_counts_filename, sep="\t")

In [ ]:
# # one-step count_barcodes function
# # warning: less efficient since barcode-UMI mapping is done per sample, rather than a single time after all sequences are counted

# def count_barcodes(fastq_file, barcode_table, umi_list, 
#                    min_seq_length=20, barcode_length=12, umi_length=9, linker_seq='GCATGC'):
#     '''
#     parses fastq file to count sequences, then extracts barcode-UMI info from sequences and assigns ID based
#     on provided reference barcode_table and umi_list. Finally, consolidates counts by assigned ID and returns
#     a dict of barcode counts.
#     does NOT do UMI mapping if umi_list is empty.
#     '''
#     # set max_seq_length
#     max_seq_length = barcode_length+umi_length+len(linker_seq) # this could be adjusted to allow insertions
#     # alphabetical sort umi_list
#     umi_list = sorted(umi_list)
    
#     # count raw sequences
#     seq_counts = count_seqs(fastq_file, min_seq_length, max_seq_length)
    
#     # consolidate barcode counts
#     barcode_counts_dict = {}
#     for seq, count in seq_counts.items():
#         # assign barcode-UMI ID
#         assigned_id = map_seq_to_barcode_umi(seq, barcode_table, umi_list, barcode_length, umi_length, linker_seq)
#         if assigned_id:
#             try:
#                 barcode_counts_dict[assigned_id] += count
#             except KeyError:
#                 barcode_counts_dict[assigned_id] = count
    
#     return barcode_counts_dict


# fastq_dict = dict(zip(sample_name_list, fastq_list))
# with mp.Pool(min(len(os.sched_getaffinity(0)), len(fastq_list))) as pool:
#     all_counts_df = {sample_name : pool.apply_async(count_barcodes, (fastq_file, barcode_table, umi_list)) for sample_name, fastq_file in fastq_dict.items()}
#     all_counts_df = {sample_name : res.get() for sample_name, res in all_counts_df.items()}
    
# # write all counts to output file
# os.makedirs(output_dir, exist_ok=True)
# all_counts_df = pd.DataFrame.from_dict(all_counts_df, orient="columns")
# all_counts_df.index.name = 'barcode'
# all_counts_df.to_csv(output_dir+counts_filename, sep="\t")